In [ ]:
from agents import Agent, Runner, function_tool, ItemHelpers

@function_tool
def get_weather(city: str):
    """Get the weather in a given city"""
    # print(f"Getting weather for {city}")
    return "30도 맑음"

agent = Agent(
    name="Assistant Agent",
    instructions="You are a helpful assistant. User tools when need to answer questions",
    tools=[get_weather],
)

stream = Runner.run_streamed(agent, "안녕 잘 지내니? 한국의 날씨는 어떄?")

async for event in stream.stream_events():
    if event.type == "raw_response_event":
        continue
    elif event.type == "agent_updated_stream_event":
        print("에이전트 업데이트 ▶ ", event.new_agent.name)
    elif event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            print(event.item.raw_item.to_dict())
        elif event.item.type == "tool_call_output_item":
            print(event.item.output)
        elif event.item.type == "message_output_item":
            print(ItemHelpers.text_message_output(event.item) )

    
    # print(event.type)
    print("="*20)


에이전트 업데이트 ▶  Assistant Agent
{'arguments': '{"city":"서울"}', 'call_id': 'call_hMy8dESmQDbduuoEbxSsLPZf', 'name': 'get_weather', 'type': 'function_call', 'id': 'fc_0d7826e9a79f0efb0068dbdceddcec819188f13607f5a8ed91', 'status': 'completed'}
Getting weather for 서울
30도 맑음
안녕하세요! 저는 항상 잘 지내고 있어요. 
오늘 한국(서울)의 날씨는 30도로 맑습니다. 무더운 여름 날씨니 시원하게 보내세요!


In [ ]:
from agents import Agent, Runner, function_tool, ItemHelpers

@function_tool
def get_weather(city: str):
    """Get the weather in a given city"""
    # print(f"Getting weather for {city}")
    return "30도 맑음"

agent = Agent(
    name="Assistant Agent",
    instructions="You are a helpful assistant. User tools when need to answer questions",
    tools=[get_weather],
)

stream = Runner.run_streamed(agent, "안녕 잘 지내니? 한국의 날씨는 어떄?")

message = ""
args = ""

async for event in stream.stream_events():
    if event.type == "raw_response_event":
        event_type = event.data.type

        if event_type == "response.output_text.delta":
            message += event.data.delta
            print(message)
        elif event_type == "response.function_call_arguments.delta":
            args += event.data.delta
            print(args)
        elif event_type == "response.completed":
            message = ""
            args = ""
    


{"
{"city
{"city":"
{"city":"Se
{"city":"Seoul
{"city":"Seoul"}
Getting weather for Seoul
안
안녕하세요
안녕하세요!
안녕하세요! 저는
안녕하세요! 저는 항상
안녕하세요! 저는 항상 잘
안녕하세요! 저는 항상 잘 지
안녕하세요! 저는 항상 잘 지내
안녕하세요! 저는 항상 잘 지내고
안녕하세요! 저는 항상 잘 지내고 있습니다
안녕하세요! 저는 항상 잘 지내고 있습니다.

안녕하세요! 저는 항상 잘 지내고 있습니다.
한국
안녕하세요! 저는 항상 잘 지내고 있습니다.
한국(
안녕하세요! 저는 항상 잘 지내고 있습니다.
한국(서울
안녕하세요! 저는 항상 잘 지내고 있습니다.
한국(서울)의
안녕하세요! 저는 항상 잘 지내고 있습니다.
한국(서울)의 현재
안녕하세요! 저는 항상 잘 지내고 있습니다.
한국(서울)의 현재 날
안녕하세요! 저는 항상 잘 지내고 있습니다.
한국(서울)의 현재 날씨
안녕하세요! 저는 항상 잘 지내고 있습니다.
한국(서울)의 현재 날씨는
안녕하세요! 저는 항상 잘 지내고 있습니다.
한국(서울)의 현재 날씨는 맑
안녕하세요! 저는 항상 잘 지내고 있습니다.
한국(서울)의 현재 날씨는 맑고
안녕하세요! 저는 항상 잘 지내고 있습니다.
한국(서울)의 현재 날씨는 맑고,
안녕하세요! 저는 항상 잘 지내고 있습니다.
한국(서울)의 현재 날씨는 맑고, 기
안녕하세요! 저는 항상 잘 지내고 있습니다.
한국(서울)의 현재 날씨는 맑고, 기온
안녕하세요! 저는 항상 잘 지내고 있습니다.
한국(서울)의 현재 날씨는 맑고, 기온은
안녕하세요! 저는 항상 잘 지내고 있습니다.
한국(서울)의 현재 날씨는 맑고, 기온은 
안녕하세요! 저는 항상 잘 지내고 있습니다.
한국(서울)의 현재 날씨는 맑고, 기온은 30
안녕하세요! 저는 항상 잘 지내고 있습니다.
한국(서울)의 현재 날씨는 맑고, 기온은 30도
안녕하세요! 저는 항상 잘 지내고 있습니다.
한국(서울)의 현재 날씨는 맑고, 기온은 30도입니다
안녕

In [ ]:
# SQLite 메모리 세션 구현
from agents import Agent, Runner, function_tool, ItemHelpers, SQLiteSession

session = SQLiteSession("user2", "ai-memory.db")

@function_tool
def get_weather(city: str):
    """Get the weather in a given city"""
    # print(f"Getting weather for {city}")
    return "30도 맑음"

agent = Agent(
    name="Assistant Agent",
    instructions="You are a helpful assistant. User tools when need to answer questions",
    tools=[get_weather],
)




In [23]:
result = await Runner.run(agent, "내 이름은 뭔가요?", session=session)

print(result.final_output)

죄송하지만, 아직 당신의 이름에 대한 정보를 받지 못했습니다. 이름을 알려주시면 그에 맞춰 도와드릴 수 있습니다!


In [6]:
# SQLite 메모리 세션 구현
from agents import Agent, Runner, ItemHelpers, SQLiteSession

session = SQLiteSession("user3", "ai-memory.db")

geaography_agent = Agent(
    name="Geo Expert Agent",
    instructions="You are a expert in geography, you answer questions related to them.",
    handoff_description="Use this to answer questions related to geography."  # 메인 agent에게 전달할 이 agent에 대한 설명
)

economics_agent = Agent(
    name="Economics Expert Agent",
    instructions="You are a expert in economics, you answer questions related to them.",
    handoff_description="Use this to answer questions related to economics."
)

main_agent = Agent(
    name="Assistant Agent",
    instructions="You are a user facing agent. Transfer to the agent monst capable of answering the user's question.",
    handoffs=[
        economics_agent,
        geaography_agent
    ]
)




In [ ]:
result = await Runner.run(
    main_agent,
    input="한국 주식 시장에 대해서 알려주세요",
    session=session,
)

print(result.last_agent.name)
print(result.final_output)

Geo Expert Agent
대한민국 수도인 서울특별시의 대표적인 GPS 좌표는  
**위도 37.5665° N, 경도 126.9780° E** 입니다.

이 좌표는 서울 시청 근처를 기준으로 한 대표값입니다.
